In [1]:
import torch

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.5839, 0.2083, 0.5952],
        [0.9532, 0.3039, 0.0214],
        [0.4746, 0.4599, 0.8636],
        [0.6760, 0.6653, 0.8332],
        [0.1365, 0.0204, 0.3924]])


# Importing needed libraries

In [51]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
import argparse
import os
import matplotlib.image as mpimg
import torchvision, torch
from torchvision import datasets, models, transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from skimage import io, transform
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time
np.random.seed(0)
print('done')

done


In [35]:
print(torch.cuda.is_available())

True


# Dataloader

In [52]:
class cam (torch.utils.data.Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.frames = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.img_name= self.frames.iloc[:,0]
        self.transform = transform
        self.steering  = self.frames.iloc[:, 3]
        self.throttle  = self.frames.iloc[:, 4]

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, index):
        img=Image.open(os.path.join(self.root_dir,
                                self.img_name[index]))

        if self.transform:
            center_camera = self.transform(img)
        steering  = self.steering [index]
        throttle = self.throttle [index]
        return center_camera, steering, throttle

# Data loading

In [56]:
transform_car = transforms.Compose([transforms.ToTensor()])
train_data = cam(csv_file="data\driving_log.csv",root_dir = "data\IMG",transform=transform_car )
train_loader=DataLoader(dataset=train_data,batch_size=32,drop_last=True,shuffle=True,num_workers=0)

In [57]:
valid_data =cam(csv_file="data_valid\driving_log.csv",root_dir = "data_valid\IMG",transform=transform_car )
valid_loader=DataLoader(dataset=valid_data,batch_size=32,shuffle=False,num_workers=0)

# Modeling

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs=10
batch_size=32
learning_rate=0.001

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.resize = transforms.Resize((224, 224), antialias=True)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size= 5, stride=2) 
        self.conv2 = nn.Conv2d(24, 36, 5, 2) 
        self.conv3 = nn.Conv2d(36, 48, 5, 2) 
        self.conv4 = nn.Conv2d(48, 64, 3, 1)
        self.conv5 = nn.Conv2d(64, 64, 3, 1)   #img_size = 28224
        self.dropout = nn.Dropout(0.5)
        self.lin1 = nn.Linear(28224, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 2)
    def forward(self, out):
        out = self.resize(out)
        out = F.elu(self.conv1(out))   
        out = F.elu(self.conv2(out))
        out = F.elu(self.conv3(out))
        out = F.elu(self.conv4(out))
        out = F.elu(self.conv5(out))
        out = self.dropout(out)
        out = out.view(out.size(0), -1) # This flattens the output of the conv layers
        out = F.elu(self.lin1(out))
        out = F.elu(self.lin2(out))
        out = F.elu(self.lin3(out))
        out = self.lin4(out)
        return out

model = ConvNet().to(device)

criterion=nn.MSELoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

print("done")

done


# Model Training

In [80]:
def training(model, num_epochs, train_loader, valid_loader, optimizer, device):
    start_time = time.time()
    min_valid_loss = 8
    minibatch_loss_list, train_acc_list = [], []
    for epoch in range(num_epochs):

        model.train()
        for batch_idx, data in enumerate(train_loader):

            center_camera, steering, throttle = data
            center_camera = center_camera.to(device)
            steering = steering.to(device)
            throttle = throttle.to(device)
            

            # Forward pass
            logits1,logits2= torch.split(model(center_camera), 1, dim =1) 
            loss1 = criterion(logits1, steering.to(torch.float32))
            loss2 = criterion(logits2, throttle.to(torch.float32))
            loss = loss1 + loss2
            
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            minibatch_loss_list.append(loss.item())
            if not batch_idx % 50:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}')

        valid_loss = 0.0
        model.eval()    
        for batch_idx_val, data_val in enumerate(valid_loader):

            center_camera_val,steering_val,throttle_val = data_val
            center_camera_val = center_camera_val.to(device)
            steering_val = steering_val.to(device)
            throttle_val = throttle_val.to(device)
            

            logits1_val,logits2_val = torch.split(model(center_camera_val), 1,dim =1)
            loss1_val = criterion(logits1_val, steering_val.to(torch.float32))
            loss2_val = criterion(logits2_val,throttle_val.to(torch.float32))
            valid_loss = loss1_val + loss2_val
            
            valid_acc_list.append(valid_loss.item())
            print(f'Epoch {epoch+1:03d}/{num_epochs:03d} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(model, 'saved_model.pth')

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    #elapsed = (time.time() - start_time)/60
    #print(f'Total Training Time: {elapsed:.2f} min')

    return minibatch_loss_list, valid_acc_list

In [81]:
minibatch_loss_list,valid_acc_list=training(model=model,
                                num_epochs=50,
                                train_loader=train_loader,
                                valid_loader=valid_loader,
                                optimizer=optimizer,
                                device=device)

Epoch: 001/050 | Batch 0000/0103 | Loss: 0.0169
Epoch: 001/050 | Batch 0050/0103 | Loss: 0.0211
Epoch: 001/050 | Batch 0100/0103 | Loss: 0.0770
Epoch 001/050 		 Validation Loss: 0.0011290046386420727
Validation Loss Decreased(8.000000--->0.119674) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.00016645388677716255
Validation Loss Decreased(0.119674--->0.017644) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.00012704674736596644
Validation Loss Decreased(0.017644--->0.013467) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.0001005257508950308
Validation Loss Decreased(0.013467--->0.010656) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 6.880643195472658e-05
Validation Loss Decreased(0.010656--->0.007293) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 3.431933146202937e-05
Validation Loss Decreased(0.007293--->0.003638) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 2.990539178426843e-05
Validation Loss Decreased(0.003638--->0.003170) 	 Saving The

C:\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 002/050 | Batch 0050/0103 | Loss: 0.0109
Epoch: 002/050 | Batch 0100/0103 | Loss: 0.0520
Epoch 002/050 		 Validation Loss: 0.001101897214539349
Epoch 002/050 		 Validation Loss: 0.00014396097685676068
Epoch 002/050 		 Validation Loss: 0.00010515334724914283
Epoch 002/050 		 Validation Loss: 8.502864511683583e-05
Epoch 002/050 		 Validation Loss: 5.4836222261656076e-05
Epoch 002/050 		 Validation Loss: 2.539811612223275e-05
Epoch 002/050 		 Validation Loss: 2.223057708761189e-05
Epoch 002/050 		 Validation Loss: 3.0290844733826816e-05
Epoch 002/050 		 Validation Loss: 4.2311152355978265e-05
Epoch 002/050 		 Validation Loss: 1.9601202438934706e-05
Epoch 002/050 		 Validation Loss: 7.073765300447121e-05
Epoch 002/050 		 Validation Loss: 0.00011122606520075351
Epoch 002/050 		 Validation Loss: 0.00020878235227428377
Epoch 002/050 		 Validation Loss: 0.00013744158786721528
Epoch 002/050 		 Validation Loss: 7.208324677776545e-05
Epoch 002/050 		 Validation Loss: 4.057721889694221e-05


Epoch 003/050 		 Validation Loss: 2.444858728267718e-05
Epoch 003/050 		 Validation Loss: 1.7628888599574566e-05
Epoch 003/050 		 Validation Loss: 3.6285400710767135e-05
Epoch 003/050 		 Validation Loss: 6.388859765138477e-05
Epoch 003/050 		 Validation Loss: 0.0002027282607741654
Epoch 003/050 		 Validation Loss: 0.00013420592586044222
Epoch 003/050 		 Validation Loss: 6.843741721240804e-05
Epoch 003/050 		 Validation Loss: 6.386703171301633e-05
Epoch 003/050 		 Validation Loss: 4.718749551102519e-06
Epoch 003/050 		 Validation Loss: 4.713215275842231e-06
Epoch 003/050 		 Validation Loss: 5.062508716946468e-05
Epoch 003/050 		 Validation Loss: 0.0003083368646912277
Epoch 003/050 		 Validation Loss: 0.00032791480771265924
Epoch 003/050 		 Validation Loss: 1.3308355846675113e-05
Epoch 003/050 		 Validation Loss: 0.0005302102072164416
Epoch 003/050 		 Validation Loss: 0.0001792854309314862
Epoch 003/050 		 Validation Loss: 3.187002948834561e-05
Epoch 003/050 		 Validation Loss: 0.0002553

Epoch 004/050 		 Validation Loss: 0.00011031404574168846
Epoch 004/050 		 Validation Loss: 5.4507210734300315e-05
Epoch 004/050 		 Validation Loss: 3.6460151022765785e-05
Epoch 004/050 		 Validation Loss: 2.6075598725583404e-05
Epoch 004/050 		 Validation Loss: 5.499942199094221e-05
Epoch 004/050 		 Validation Loss: 2.8892816771985963e-05
Epoch 004/050 		 Validation Loss: 5.0002316129393876e-05
Epoch 004/050 		 Validation Loss: 6.0851729358546436e-05
Epoch 004/050 		 Validation Loss: 8.054120553424582e-05
Epoch 004/050 		 Validation Loss: 0.00017683417536318302
Epoch 004/050 		 Validation Loss: 0.0002681411860976368
Epoch 004/050 		 Validation Loss: 0.0004768040671478957
Epoch 004/050 		 Validation Loss: 5.580057404586114e-05
Epoch 004/050 		 Validation Loss: 7.988927245605737e-05
Epoch 004/050 		 Validation Loss: 0.0004947060951963067
Epoch 004/050 		 Validation Loss: 1.835521834436804e-05
Epoch 004/050 		 Validation Loss: 0.00010098532948177308
Epoch 004/050 		 Validation Loss: 0.000

...

Epoch 045/050 		 Validation Loss: 0.0028390109073370695
Epoch 045/050 		 Validation Loss: 0.0014240076998248696
Epoch 045/050 		 Validation Loss: 0.0016152948373928666
Epoch 045/050 		 Validation Loss: 0.0006357850506901741
Epoch 045/050 		 Validation Loss: 0.0004487823462113738
Epoch 045/050 		 Validation Loss: 0.0003545363724697381
Epoch 045/050 		 Validation Loss: 0.0006754224305041134
Epoch 045/050 		 Validation Loss: 0.0011743679642677307
Epoch 045/050 		 Validation Loss: 0.0017226438503712416
Epoch 045/050 		 Validation Loss: 0.0007054614834487438
Epoch 045/050 		 Validation Loss: 0.0005890193860977888
Epoch 045/050 		 Validation Loss: 0.0005887750303372741
Epoch 045/050 		 Validation Loss: 0.0007737954729236662
Epoch 045/050 		 Validation Loss: 0.0007641652482561767
Epoch 045/050 		 Validation Loss: 0.001050615101121366
Epoch 045/050 		 Validation Loss: 0.0012564118951559067
Epoch 045/050 		 Validation Loss: 0.0008818659698590636
Epoch 045/050 		 Validation Loss: 0.0007137487409

Epoch 046/050 		 Validation Loss: 0.0008327470277436078
Epoch 046/050 		 Validation Loss: 0.0010288592893630266
Epoch 046/050 		 Validation Loss: 0.0008023647242225707
Epoch 046/050 		 Validation Loss: 0.0006461029988713562
Epoch 046/050 		 Validation Loss: 0.0005321296630427241
Epoch 046/050 		 Validation Loss: 0.0006025312468409538
Epoch 046/050 		 Validation Loss: 0.0005588209605775774
Epoch 046/050 		 Validation Loss: 0.000568078481592238
Epoch 046/050 		 Validation Loss: 0.0005931872874498367
Epoch 046/050 		 Validation Loss: 0.0006519724847748876
Epoch 046/050 		 Validation Loss: 0.0007018428295850754
Epoch 046/050 		 Validation Loss: 0.0007799345767125487
Epoch 046/050 		 Validation Loss: 0.0010177328949794173
Time elapsed: 8.82 min
Epoch: 047/050 | Batch 0000/0103 | Loss: 0.0705
Epoch: 047/050 | Batch 0050/0103 | Loss: 0.0524
Epoch: 047/050 | Batch 0100/0103 | Loss: 0.0875
Epoch 047/050 		 Validation Loss: 0.0008595676627010107
Epoch 047/050 		 Validation Loss: 0.00028757448308

Epoch 048/050 		 Validation Loss: 0.0003090568061452359
Epoch 048/050 		 Validation Loss: 0.0015316802309826016
Epoch 048/050 		 Validation Loss: 0.0004680718411691487
Epoch 048/050 		 Validation Loss: 0.0002586345071904361
Epoch 048/050 		 Validation Loss: 0.00023054135090205818
Epoch 048/050 		 Validation Loss: 0.00020104096620343626
Epoch 048/050 		 Validation Loss: 0.0002139363787136972
Epoch 048/050 		 Validation Loss: 0.00022446380171459168
Epoch 048/050 		 Validation Loss: 0.00024163236957974732
Epoch 048/050 		 Validation Loss: 0.0002609285875223577
Epoch 048/050 		 Validation Loss: 0.0002673982526175678
Epoch 048/050 		 Validation Loss: 0.0002865768037736416
Epoch 048/050 		 Validation Loss: 0.00026036149938590825
Epoch 048/050 		 Validation Loss: 0.0002665588690433651
Epoch 048/050 		 Validation Loss: 0.0002519706904422492
Epoch 048/050 		 Validation Loss: 0.0002314462763024494
Epoch 048/050 		 Validation Loss: 0.00021870715136174113
Epoch 048/050 		 Validation Loss: 0.000170

Epoch 049/050 		 Validation Loss: 0.0011141154682263732
Epoch 049/050 		 Validation Loss: 0.00031812238739803433
Epoch 049/050 		 Validation Loss: 0.0002348708367208019
Epoch 049/050 		 Validation Loss: 0.0002321778010809794
Epoch 049/050 		 Validation Loss: 0.00036902472493238747
Epoch 049/050 		 Validation Loss: 0.0003622099757194519
Epoch 049/050 		 Validation Loss: 0.0005778059712611139
Epoch 049/050 		 Validation Loss: 0.0007456374587491155
Epoch 049/050 		 Validation Loss: 0.0004524588875938207
Epoch 049/050 		 Validation Loss: 0.0003279426309745759
Epoch 049/050 		 Validation Loss: 0.0003136179584544152
Epoch 049/050 		 Validation Loss: 0.00028182074311189353
Epoch 049/050 		 Validation Loss: 0.0003033932007383555
Epoch 049/050 		 Validation Loss: 0.00029375270241871476
Epoch 049/050 		 Validation Loss: 0.00033558852737769485
Epoch 049/050 		 Validation Loss: 0.0003578604373615235
Epoch 049/050 		 Validation Loss: 0.0004090848669875413
Epoch 049/050 		 Validation Loss: 0.0005584

Epoch 050/050 		 Validation Loss: 0.00025285728042945266
Epoch 050/050 		 Validation Loss: 0.0002696489973459393
Epoch 050/050 		 Validation Loss: 0.0003163890796713531
Epoch 050/050 		 Validation Loss: 0.00034895961289294064
Epoch 050/050 		 Validation Loss: 0.00040307227754965425
Epoch 050/050 		 Validation Loss: 0.000597693317104131
Time elapsed: 9.57 min
